In [ ]:
from pyniryo2 import *
import pyniryo
from matplotlib import pyplot as plt
import cv2 as cv
import time
import random
from motion import movement
from motion import clear_out
from postions_played import positions_played
import time
from pyniryo2 import *
import pyniryo
from matplotlib import pyplot as plt
import cv2 as cv
import time
from motion import pickup_coordinates
from motion import dropoff_coordinates

# Connect to robot & calibrate
#robot = NiryoRobot("10.10.10.10")
robot = NiryoRobot("169.254.200.200")
robot.arm.calibrate_auto()
from recorded_positions import pickup_coordinates,dropoff_coordinates

In [ ]:
robot_ip_address = "169.254.200.200"
robot = NiryoRobot(robot_ip_address)
def get_pickup_coordinates(p):
    return pickup_coordinates.get(p,'Invalid_position')
robot.arm.move_pose(get_pickup_coordinates('h0'))

In [ ]:
class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]

    def is_winner(self, player):
        win_conditions = [
            (0, 1, 2),
            (3, 4, 5),
            (6, 7, 8),
            (0, 3, 6),
            (1, 4, 7),
            (2, 5, 8),
            (0, 4, 8),
            (2, 4, 6)
        ]
        for condition in win_conditions:
            if all(self.board[i] == player for i in condition):
                return True
        return False

    def available_moves(self):
        return [i for i in range(9) if self.board[i] == " "]

    def make_move(self, position, player):
        if self.board[position] == " ":
            self.board[position] = player
            return True
        return False


In [ ]:
class MinimaxAgent:
    def __init__(self, game):
        self.game = game

    def minimax(self, board, depth, is_maximizing):
        if self.game.is_winner("X"):
            return -1
        elif self.game.is_winner("O"):
            return 1
        elif len(self.game.available_moves()) == 0:
            return 0

        if is_maximizing:
            best_score = -float('inf')
            for move in self.game.available_moves():
                board[move] = "O"
                score = self.minimax(board, depth + 1, False)
                board[move] = " "
                best_score = max(score, best_score)
            return best_score
        else:
            best_score = float('inf')
            for move in self.game.available_moves():
                board[move] = "X"
                score = self.minimax(board, depth + 1, True)
                board[move] = " "
                best_score = min(score, best_score)
            return best_score

    def get_move(self):
        best_move = None
        best_score = -float('inf')
        for move in self.game.available_moves():
            self.game.board[move] = "O"
            score = self.minimax(self.game.board, 0, False)
            self.game.board[move] = " "
            if score > best_score:
                best_score = score
                best_move = move
        return best_move


In [ ]:

class RandomAgent:
    def __init__(self, game):
        self.game = game  

    def get_move(self):
        return random.choice(self.game.available_moves())

In [ ]:
def choose_agent():
    print("Choose an agent to play against:")
    print("1. Minimax Agent (Hard)")
    print("2. Random Agent (Easy)")
    print("3. Hybrid Agent (Minimax for 2 moves, then Random)")
    choice = input("Enter 1, 2, or 3: ")
    
    if choice == "1":
        return "minimax"
    elif choice == "2":
        return "random"
    elif choice == "3":
        return "hybrid"
    else:
        print("Invalid choice, defaulting to Random Agent.")
        return "random"

In [ ]:


def get_pickup_coordinates(p):
    return pickup_coordinates.get(p,'Invalid_position')
def get_dropoff_coordinates(p):
    return dropoff_coordinates.get(p,'Invalid_position')
def go_up(position_coordinates):
    coordinates=[0,0,0.03 ,0,0,0]
    result = [a + b for a, b in zip(position_coordinates, coordinates)]
    robot.arm.move_pose(result)

def go_down(position_coordinates):
    coordinates=[0,0,0.03 ,0,0,0]
    result = [a + b for a, b in zip(position_coordinates, coordinates)]
    robot.arm.move_pose(result)

In [ ]:
def play_move(pick_up_list,agent_move):
    position_coordinates = get_pickup_coordinates(pick_up_list[0])
    robot.arm.move_pose(position_coordinates)
    robot.tool.grasp_with_tool()
    go_up(position_coordinates)

    position_coordinates = get_dropoff_coordinates(f'p{agent_move + 1}')
    robot.arm.move_pose(position_coordinates)
    go_down(position_coordinates)
    robot.arm.move_pose(position_coordinates)
    robot.tool.release_with_tool()

    pick_up_list.pop(0)

def choose_agent():
    print("Choose an agent to play against:")
    print("1. Minimax Agent (Hard)")
    print("2. Random Agent (Easy)")
    print("3. Hybrid Agent (Minimax for 2 moves, then Random)")
    choice = input("Enter\n1 for minmax agent.\n2 for random agent.\n3 for hybrid. ")
    
    if choice == "1":
        return "minimax"
    elif choice == "2":
        return "random"
    elif choice == "3":
        return "hybrid"
    else:
        return "random"



In [9]:

import sounds

def play_game():
    game = TicTacToe()
    agent_type = choose_agent()

    if agent_type == "minimax":
        agent = MinimaxAgent(game)
    elif agent_type == "random":
        agent = RandomAgent(game)
    else:
        agent = MinimaxAgent(game)
        hybrid_switch = 2

    global player_positions_stored
    player_positions_stored = []
    global robot_positions_stored
    robot_positions_stored = []
    global pick_up_list
    pick_up_list = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10']
    global pickup_index
    pickup_index = 0

    r=input('Game Type:\n 1. robot vs robot \n 2. Robot vs player')
    feild_clean = 1
    first_player = input("Who will play first? Enter '1' for Robot or '2' for Player: ")
    if r is not first_player:
        feild_clean=2
    sounds.startsound()
    while True:
        if first_player == '1':
            if agent_type == "hybrid" and hybrid_switch == 0:
                agent = RandomAgent(game)

            agent_move = agent.get_move()
            #time.sleep(5)
            movement(agent_move + 1, pickup_index=pickup_index)
            game.make_move(agent_move, "O")
            print(f"Agent plays at position: p{agent_move + 1}")
            robot_positions_stored.append(f'p{agent_move + 1}')
            pickup_index += 1

            if agent_type == "hybrid" and hybrid_switch > 0:
                hybrid_switch -= 1

            if game.is_winner("O"):
                print("Agent wins!")
                sounds.endgamesound()
                clear_out(robot_positions_stored, player_positions_stored)
                break
            elif len(game.available_moves()) == 0:
                print("It's a tie!")
                sounds.endgamesound()
                clear_out(robot_positions_stored, player_positions_stored)
                break

            first_player = '2'

        players_input = positions_played()

        if len(players_input) > 0:
            position_input = players_input[-1]
        print('Player plays at', position_input)

        position = int(position_input[1]) - 1
        if not game.make_move(position, "X"):
            print("Invalid move! Try again.")
            continue
        player_positions_stored.append(f'p{position_input[-1]}')

        if game.is_winner("X"):
            print("You win!")
            sounds.endgamesound()
            print('Player position list', player_positions_stored)
            clear_out(robot_positions_stored, player_positions_stored,feild_clean)
            break
        elif len(game.available_moves()) == 0:
            print("It's a tie!")
            sounds.endgamesound()
            clear_out(robot_positions_stored, player_positions_stored,feild_clean)
            break

        if agent_type == "hybrid" and hybrid_switch == 0:
            agent = RandomAgent(game)

        agent_move = agent.get_move()
        movement(agent_move + 1, pickup_index=pickup_index)
        game.make_move(agent_move, "O")
        print(f"Agent plays at position: p{agent_move + 1}")
        robot_positions_stored.append(f'p{agent_move + 1}')
        pickup_index += 1

        if agent_type == "hybrid" and hybrid_switch > 0:
            hybrid_switch -= 1

        if game.is_winner("O"):
            print("Agent wins!")
            sounds.endgamesound()
            clear_out(robot_positions_stored, player_positions_stored,feild_clean)
            break
        elif len(game.available_moves()) == 0:
            print("It's a tie!")
            sounds.endgamesound()
            clear_out(robot_positions_stored, player_positions_stored,feild_clean)
            break

    print("Final positions:")
    print("Player:", player_positions_stored)
    print("Robot:", robot_positions_stored)

play_game()


KeyboardInterrupt: 